In [1]:
#Stitch is a function that stitches 2 images together,where 1st input argument should be left image 
#and the second argument should be right image
def Stitch(imgL,imgR,iteration):      
        p1=[]
        p2=[]
        img1=cv2.cvtColor(imgL,cv2.COLOR_BGR2GRAY)
        img2=cv2.cvtColor(imgR,cv2.COLOR_BGR2GRAY)
        sift=cv2.xfeatures2d.SIFT_create()
        kp1,des1 =sift.detectAndCompute(img1,None)
        kp2,des2 =sift.detectAndCompute(img2,None)
        cv2.imwrite('image_mosaicing/images6/left image_keypoints'+str(iteration)+'.jpg',cv2.drawKeypoints(img1,kp1,None))
        cv2.imwrite('image_mosaicing/images6/right image_keypoints'+str(iteration)+'.jpg',cv2.drawKeypoints(img2,kp2,None))
        match = cv2.BFMatcher()
        matches = match.knnMatch(des1,des2,k=2)
        good = []
        for m,n in matches:
            if m.distance < 0.5*n.distance:
                good.append(m)

        if len(good)>10:
            draw_params = dict(matchColor = (0,0,255), # draw matches in red color
                               singlePointColor = None,
                               flags = 2)

            img3 = cv2.drawMatches(img1,kp1,img2,kp2,good,None,**draw_params)
            cv2.imwrite('image_mosaicing/images6/drawMatches' +str(iteration) +'.jpg', img3)

            for i in range(len(good)):
                idx1=good[i].queryIdx
                idx2=good[i].trainIdx
                p1.append(kp1[idx1].pt)
                p2.append(kp2[idx2].pt)
            p1=np.asarray(p1)
            p2=np.asarray(p2)
            H=RANSAC(p1,p2)
            dst = cv2.warpPerspective(imgR,H,(imgR.shape[1]+imgL.shape[1], imgL.shape[0]+imgR.shape[0]))
            cv2.imwrite('image_mosaicing/images6/warped' +str(iteration) +'.jpg', dst)
            dst[0:imgL.shape[0], 0:imgL.shape[1]] = imgL
            crop_im=trim(dst)
            return(crop_im)
        else:
            print('Not enough matches found')
            return(None)

In [2]:
#This function crops the intermdiate results
def trim(img):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    _,thresh = cv2.threshold(gray,2,255,cv2.THRESH_BINARY)
    contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]
    for i in contours:
        x,y,w,h = cv2.boundingRect(i)
        crop = img[y:y+h,x:x+w]
    return(crop)    



In [3]:
#RANSAC function calculates the Homography between the 2 images given the correspondence
def RANSAC(p1,p2):    
    from random import sample
    RANSAC_iter=100
    num_points =[i for i in range(p1.shape[0])]
    diff_norm=np.empty((RANSAC_iter,1))
    diff=np.empty((p1.shape[0],1))
    M = np.empty((8,9))
    H12=np.empty((RANSAC_iter,3,3))
    for k in range(RANSAC_iter):
        four_p = sample(num_points, 4)
        j=0
        for i in four_p:
            # u2,v2,1,0,0,0,-u1u2,-u1v2,-u1 :u2=p2x,v2=p2y,u1=p1x,v1=p1y
            # 0,0,0,u2,v2,1,-v1u2,-v1v2,-v1
            M[j,:]=[p2[i,0],p2[i,1],1,0,0,0,-p1[i,0]*p2[i,0],-p1[i,0]*p2[i,1],-p1[i,0]]
            M[j+1,:]=[0,0,0,p2[i,0],p2[i,1],1,-p1[i,1]*p2[i,0],-p1[i,1]*p2[i,1],-p1[i,1]]
            j+=2
        (U, S, V) = np.linalg.svd(M, full_matrices=True)
        f = V[-1,:]
        H12[k] =np.reshape(f,(3,3))
        for i in range(p1.shape[0]):
                p2_=np.array([p2[i,0],p2[i,1],1])
                p1_=np.matmul(H12[k],p2_)
                p1_=p1_/(p1_[2]+0.000000001)
                diff[i]=np.linalg.norm(p1[i]-p1_[0:2])
        diff_norm[k]=np.sum(diff)/p1.shape[0]
    idx_min= np.argmin(diff_norm)
    H=H12[idx_min]    
    H=H/H[2,2]
    print(H)
    print('MinError in H',diff_norm[idx_min])
    return(H)

In [4]:
#The Main Code 
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from os import listdir
from os.path import isfile, join
import glob
import sys
#all images that are to be stitched are kept in separate folder together and output of each
#stitching iteration are stored back in the same folder
#The images must be in the order of (TOP to BOTTOM) from (LEFT MOST to RIGHT MOST)
mypath='C:\\Users\\Asus\\Computer Vision\\Assignment2\\image_mosaicing\\images6'
onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
images = np.empty(len(onlyfiles), dtype=object)
out = np.empty(len(onlyfiles)-1, dtype=object)
for n in range(len(onlyfiles)):
    images[n] = cv2.imread( join(mypath,onlyfiles[n]) )
j=0    
if len(images)%2==0:       #if there are even number of images
    for i in range(0,len(images),2):
        out[j]=Stitch(images[i],images[i+1],i)
        if out[j].any()==None:
            print('Noisy image present')
            sys.exit()
        cv2.imwrite('image_mosaicing/images6/output_im'+str(j)+'.jpg',out[j])
        j+=1
    if(out.shape[0]>1):
        out[j]=Stitch(out[0],out[1],j)
        cv2.imwrite('image_mosaicing/images6/output_im'+str(j)+'.jpg',out[j])    
        if(j+1<out.shape[0]):
            out[j+1]=Stitch(out[j],out[j-2],j+1)
            cv2.imwrite('image_mosaicing/images6/output_im'+str(j+1)+'.jpg',out[j+1])


else:                                #if there are odd number of images
    for i in range(0,len(images)-1,2):
        out[j]=Stitch(images[i],images[i+1],i)
        if out[j].any()==None:
            print('Noisy image present')
            sys.exit()
        cv2.imwrite('image_mosaicing/images6/output_im'+str(j)+'.jpg',out[j])
        j+=1
    
    if(out.shape[0]>1):
        out[j]=Stitch(out[0],out[1],j)
        cv2.imwrite('image_mosaicing/images6/output_im'+str(j)+'.jpg',out[j])    
        if(j+1<out.shape[0]):
            out[j+1]=Stitch(out[j],out[j-2],j+1)
            cv2.imwrite('image_mosaicing/images6/output_im'+str(j+1)+'.jpg',out[j+1])

    out[-1]=Stitch(out[-2],images[-1],out.shape[0])
    cv2.imwrite('image_mosaicing/images6/output_im'+str(j)+'.jpg',out[j])


[[ 7.60333655e-01 -5.54244677e-03  2.82248028e+02]
 [-2.06898699e-01  8.84129176e-01  8.61362265e+01]
 [-3.26141506e-04 -4.66942154e-05  1.00000000e+00]]
MinError in H [1.07665315]
